# Project 001 UCSD Data Science Visualization Bootcamp
* Noreen Malik 😃
* Rene Venema("dutch") 🐱
* John Warlop 🦁
* Alex Wooten 🏄

😛😳😗😓🙉😸🙈🙊😽💀💢💥✨💏👫👄👃👀👛👛🗼🔮🔮🎄🎅👻

## Dependencies -- imports


In [9]:
import pandas as pd
import json

## Constants -- API Keys, etc

In [10]:
# Read in API(.gitignore file has entry for api_keys file)
with open('api_keys') as json_data:
    api_keys = json.load(json_data)
print(api_keys['socrata']['App_Token'])
print(api_keys['socrata']['Secret_Token'])

47dG6YpDB3IUSlJEovXa0WCeA
3f1tMrLi0-Ck9QRZnOs9F5bi7VzkfBD6EAbL


## Functions 


In [11]:
def winning_team():
    print("Noreen, 'dutch', Alex and John make for a winning team")

## Solution Space


In [12]:
winning_team()

Noreen, 'dutch', Alex and John make for a winning team
